<a href="https://colab.research.google.com/github/Davidalencaste/Breast-cancer-screening/blob/main/Copia_de_Proyecto_David_Alencaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lectura de Datos

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from google.colab import drive
import shutil, pathlib
import os
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import GlobalAveragePooling2D
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from PIL import Image
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/ProyectoFinalRedes/data/'
#Dirección donde debe existir la carpeta (nombrada data) donde se encunetran todos los archivos

In [ ]:
calc_train = pd.read_csv(path+'csv/calc_case_description_train_set.csv')
calc_test = pd.read_csv(path+'csv/calc_case_description_test_set.csv')
mass_train = pd.read_csv(path+'csv/mass_case_description_train_set.csv')
mass_test = pd.read_csv(path+'csv/mass_case_description_test_set.csv')
dicom_df = pd.read_csv(path+'csv/dicom_info.csv')


In [ ]:
calc_test.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...


In [ ]:
calc_test["pathology"].unique()

array(['BENIGN', 'BENIGN_WITHOUT_CALLBACK', 'MALIGNANT'], dtype=object)

In [ ]:
mass_test.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,Mass-Test_P_00032_RIGHT_CC/1.3.6.1.4.1.9590.10...,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....


In [ ]:
mass_test["pathology"].unique()

array(['MALIGNANT', 'BENIGN', 'BENIGN_WITHOUT_CALLBACK'], dtype=object)

In [ ]:
#Quitar espacios en blanco en columnames y remplazar por "_"
calc_train.columns = calc_train.columns.str.replace(' ', '_')
calc_test.columns = calc_test.columns.str.replace(' ', '_')
mass_train.columns = mass_train.columns.str.replace(' ', '_')
mass_test.columns = mass_test.columns.str.replace(' ', '_')



In [ ]:
calc_train[calc_train['cropped_image_file_path'].isnull()]

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path


In [ ]:
#Solo estamos arreglando la dirección de ruta cargadas de las imagenes
a=dicom_df.image_path.apply(lambda x: x.replace(x[:9],path))
dicom_df.image_path=a

Dividimos las imagenes sus categorias: Imagenes recortadas, Imagenes Completas de Mamografia e Imágenes de máscara ROI

In [ ]:
cropp_images=dicom_df[dicom_df.SeriesDescription=='cropped images'].image_path
full_images=dicom_df[dicom_df.SeriesDescription	=='full mammogram images'].image_path
mask_images=dicom_df[dicom_df.SeriesDescription	=='ROI mask images'].image_path
cropp_images[0]

'/content/drive/MyDrive/ProyectoFinalRedes/data//jpeg/1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304/1-172.jpg'

Comenzamos a unir la base de los datos numericos con la base de imagenes

In [ ]:
def getImageFile(data):
  Ndict={}
  for dat in data:
    key=dat.split("/")[8] #Dirección del dato (1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304 ejemplo)
    Ndict[key]=dat
  return Ndict

In [ ]:
croppedDict = getImageFile(cropp_images)
mammogramDict = getImageFile(full_images)
maskDict = getImageFile(mask_images)
#croppedDict['1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304'] #Prueba

In [ ]:
#Función que une las direcciones de las imagenes con los datos en mass y calc
def data_image(data):
  for i, image in enumerate(data.values):

    imageName = image[12].split("/")[2]

    if imageName in croppedDict:

      data.iloc[i,12]=croppedDict[imageName]
    else:
      data.iloc[i,12]=None

    imageName = image[13].split("/")[2]
    if imageName in maskDict:
      data.iloc[i,13]= maskDict[imageName]
    else:
      data.iloc[i,13]=None

    imageName = image[11].split("/")[2]

    if imageName in mammogramDict:
      data.iloc[i,11]= mammogramDict[imageName]
    else:
      data.iloc[i,11]=None


In [ ]:
#mass_train.head()

In [ ]:
#Con esto cada train y test ya tiene correctamente las direcciones de las imagenes que tienen.
data_image(mass_test)
data_image(mass_train)
data_image(calc_test)
data_image(calc_train)
calc_train=calc_train.dropna(subset=['cropped_image_file_path'],how='any')
calc_test=calc_test.dropna(subset=['cropped_image_file_path'], how='any')
calc_validation=calc_test#.iloc[0:500]
calc_test#=calc_test.iloc[500:]
#mass_train.head()
#Si corres esta mas de una vez seguida obtendras un error. Deebes cargar de nuevo todo lo anterior antes de volver a cargar

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,P_02464,2,RIGHT,MLO,1,calcification,FINE_LINEAR_BRANCHING,CLUSTERED,0,MALIGNANT,4,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
322,P_02498,4,RIGHT,CC,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
323,P_02498,4,RIGHT,MLO,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
324,P_02501,3,RIGHT,CC,1,calcification,PLEOMORPHIC,CLUSTERED,0,MALIGNANT,3,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...,None


In [ ]:
print(len(calc_test))


326


In [ ]:
print(len(calc_train))

1544


In [ ]:
def display_images(dataset, column, number):
    # Crear figura y ejes
    fig, axes = plt.subplots(1, number, figsize=(15, 5))

    for index, (i, row) in enumerate(dataset.head(number).iterrows()):
        image_path = row[column]

        if image_path is None or not os.path.exists(image_path):
            # Si el archivo no existe o la ruta es inválida, omitir
            continue

        # Leer la imagen
        image = cv2.imread(image_path)

        if image is None:
            # Si hay un error al leer la imagen, omitir
            continue

        # Convertir BGR a RGB si es una imagen a color
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Seleccionar el cmap para que sea en escala de grises si es una imagen monocromática
        ax = axes[index]
        ax.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        ax.set_title(f"{row['pathology']}")
        ax.axis('off')
        print(np.array(image).shape)  # Mostrar la forma de la imagen para verificar si es RGB o escala de grises

    plt.tight_layout()
    plt.show()


In [ ]:
calc_test.head()

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,None,/content/drive/MyDrive/ProyectoFinalRedes/data...,None


In [ ]:
calc_train.head()

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...,/content/drive/MyDrive/ProyectoFinalRedes/data...


## Preparado de Datos


In [ ]:
original_dir = pathlib.Path("/content/drive/MyDrive/ProyectoFinalRedes/data/jpeg")
new_base_dir = pathlib.Path("Type_Tumor")

In [ ]:
full_data=pd.concat([calc_train,calc_test],axis =0)
len(full_data)

1870

In [ ]:
x_size=256
y_size=256
batch_size=13

### Aumentado de Datos General

In [ ]:
def augment_image(image):
    # Aplicamos aumnetado de datos usando tf.image functions
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    return image

In [ ]:
#Reescalamos imagenes
def resize_image(image_tensor):
    return tf.image.resize(image_tensor, [x_size, y_size])

In [ ]:
# Función para copiar y aumentar imágenes
from concurrent.futures import ThreadPoolExecutor # Import ThreadPoolExecutor
import os
import tensorflow as tf
from PIL import Image
import uuid
import pathlib
def process_file(src_dst):
    src, dst = src_dst
    try:
        # Abrir la imagen
        with Image.open(src) as img:
            img = img.convert("RGB")  # Convertir a formato RGB

            # Aplicar aumentos de datos y guardar versiones originales y aumentadas
            img_tensor = tf.convert_to_tensor(img)
            resized_tensor = resize_image(img_tensor)

            # Guardar la imagen original redimensionada
            original_filename = f"{uuid.uuid4().hex}_original.jpg"
            original_path = dst.parent / original_filename
            tf.keras.preprocessing.image.save_img(original_path, resized_tensor)

            # Crear y guardar imágenes aumentadas
            for i in range(3):  # Generar 3 aumentos por imagen
                augmented_tensor = augment_image(resized_tensor)
                augmented_filename = f"{uuid.uuid4().hex}_aug{i}.jpg"
                augmented_path = dst.parent / augmented_filename
                tf.keras.preprocessing.image.save_img(augmented_path, augmented_tensor)
    except Exception as e:
        print(f"Error procesando {src}: {e}")

In [ ]:
# Función para crear subconjuntos con aumentos
def make_subset(subset_name, data):
    subset_dir = new_base_dir / subset_name
    os.makedirs(subset_dir, exist_ok=True)

    # Crear directorios de categorías
    categories = ('BENIGN', 'BENIGN_WITHOUT_CALLBACK', 'MALIGNANT')
    for category in categories:
        os.makedirs(subset_dir / category, exist_ok=True)

    # Crear lista de tareas para procesamiento
    tasks = []
    paths = data[['cropped_image_file_path', 'pathology']].dropna()
    for i, (src, category) in enumerate(zip(paths.iloc[:, 0], paths.iloc[:, 1])):
        dst = subset_dir / category.strip() / f"{i}.jpg"
        tasks.append((src, dst))

    # Procesar archivos en paralelo
    with ThreadPoolExecutor() as executor:
        executor.map(process_file, tasks)

In [ ]:
make_subset("full_dataSet",full_data)

In [ ]:
zero_class_count = len(os.listdir("Type_Tumor/full_dataSet/BENIGN"))
one_class_count = len(os.listdir("Type_Tumor/full_dataSet/BENIGN_WITHOUT_CALLBACK"))
two_class_count = len(os.listdir("Type_Tumor/full_dataSet/MALIGNANT"))
print(f"Number of images in class 0: {zero_class_count}")
print(f"Number of images in class 1: {one_class_count}")
print(f"Number of images in class 2: {two_class_count}")

Number of images in class 0: 2632
Number of images in class 1: 2160
Number of images in class 2: 2688


In [ ]:
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Type_Tumor/full_dataSet",
    labels='inferred',
    label_mode='categorical',
    image_size=(x_size, y_size),
    seed=50,
    shuffle=True,
    batch_size=13
)
# Calculate the total number of samples
total_samples = tf.data.experimental.cardinality(full_dataset).numpy()

train_size = int(0.8 * total_samples)                 # 70% for training
val_size   = int(0.15 * total_samples)                # 20% for validation
test_size = total_samples - train_size - val_size     # 10% for testing

# Create train, validation, and test datasets
train_dataset       = full_dataset.take(train_size)
validation_dataset  = full_dataset.skip(train_size).take(val_size)
test_dataset        = full_dataset.skip(train_size + val_size)

train_dataset      = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset       = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print the number of samples in each dataset
print(f"Train samples:      {train_size}     batches(13) ==> {train_size*13}")
print(f"Validation samples: {val_size}       batches(13) ==> {val_size*13}")
print(f"Test samples:       {test_size}      batches(13) ==> {test_size*13}")

Found 7480 files belonging to 3 classes.
Train samples:      460     batches(13) ==> 5980
Validation samples: 86       batches(13) ==> 1118
Test samples:       30      batches(13) ==> 390


In [ ]:
from tensorflow.keras.applications import ResNet50

def try_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(x_size, y_size, 3))

    # Freeze all layers initially
    for layer in base_model.layers:
        layer.trainable = False

    # Calculate the index to start unfreezing layers
    from_index = int(np.round((len(base_model.layers) - 1) * (1.0 - 50.0 / 100.0)))

    # Unfreeze layers from the calculated index onwards
    for layer in base_model.layers[from_index:]:
        layer.trainable = True

    # Add custom layers on top (Upper-Layers)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(3, activation='softmax')(x)  # Assuming binary classification

    model = keras.Model(inputs=base_model.input, outputs=predictions)

    # # Clear the base model from memory if needed (optional)
    # del model_dict, base_model, from_index, x, predictions;    gc.collect()
    return model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam

trymodel = try_model()

trymodel.compile(optimizer=Adam(learning_rate=1e-4),
                      loss='categorical_crossentropy',
                      metrics=['accuracy', Recall(name='recall')])  # Compile the model
trymodel.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 23,463,427 (89.51 MB)

 Non-trainable params: 2,225,536 (8.49 MB)

In [ ]:
history = trymodel.fit(
            train_dataset,
            validation_data=validation_dataset,
            batch_size=13,
            epochs=7
        )

### Creamos un directorio de los datos considerando el tipo de tumor

In [ ]:
'''
original_dir = pathlib.Path("/content/drive/MyDrive/ProyectoFinalRedes/data/jpeg")
new_base_dir = pathlib.Path("Type_Tumor")

from concurrent.futures import ThreadPoolExecutor
import shutil

def copy_file(src_dst):
    src, dst = src_dst
    try:
        shutil.copy2(src, dst)
    except Exception as e:
        print(f"Error al copiar {src} a {dst}: {e}")

def make_subset(subset_name, data):
    os.makedirs(new_base_dir / subset_name, exist_ok=True)

    # Crear directorios
    categories = ('BENIGN', 'BENIGN_WITHOUT_CALLBACK', 'MALIGNANT')
    for category in categories:
        os.makedirs(new_base_dir / subset_name / category, exist_ok=True)

    # Crear lista de tareas para copiar
    tasks = []
    paths = data[['cropped_image_file_path', 'pathology']].dropna()
    for i, (src, category) in enumerate(zip(paths.iloc[:, 0], paths.iloc[:, 1])):
        dst = new_base_dir / subset_name / category.strip() / f"{i}.jpg"
        if not pathlib.Path(dst).exists():
            tasks.append((src, dst))

    # Copiar archivos en paralelo
    with ThreadPoolExecutor() as executor:
        executor.map(copy_file, tasks)


make_subset("train", data=calc_train)
#make_subset("validation", data=calc_test)
make_subset("test", data=calc_test);



In [ ]:
batchSize=64

In [ ]:
'''
train_dataset = image_dataset_from_directory(
    new_base_dir /"train",
    image_size=(x_size, y_size),
    batch_size=batchSize)

validation_dataset = image_dataset_from_directory(
    new_base_dir /"validation",
    image_size=(x_size, y_size),
    batch_size=batchSize)

test_dataset = image_dataset_from_directory(
    new_base_dir /"test",
    image_size=(x_size, y_size),
    batch_size=batchSize)


In [ ]:
train_dataset.class_names

In [ ]:
'''
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


Normalización

In [ ]:
#normalization_layer = layers.Rescaling(1./255)
#normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))

In [ ]:
'''
train_images_paths=calc_train['cropped_image_file_path']
train_labels=calc_train['pathology']
test_images_paths=calc_test['cropped_image_file_path']
test_labels=calc_test['pathology']
'''

In [ ]:
'''
# Carga de datos de entrada: Se cargan las imágenes. Las imágenes se escalan al tamaño deseado y se convierten en un arreglo NumPy.
def normaliza(ruta):
  #images = np.asarray([ np.asarray(Image.open(file).resize((500, 500))) for file in ruta])
  images = np.asarray([np.asarray(Image.open(file).convert("RGB").resize((x_size, y_size))) for file in ruta])
  print ("shape: " + str(images.shape))
  return images
'''


In [ ]:
'''
train_images = normaliza(train_images_paths)
test_images = normaliza(test_images_paths)

validation_image=test_images[:500]
validation_labels=test_labels[:500]
test_images=test_images[500:]
test_labels=test_labels[500:]
'''

In [ ]:
'''
label_mapping = {
    'MALIGNANT': 0,
    'BENIGN': 1,
    'BENIGN_WITHOUT_CALLBACK': 2
}

# Convert train_labels to numerical labels using the mapping
train_labels_numerical = np.array([label_mapping[label] for label in train_labels])
test_labels_numerical = np.array([label_mapping[label] for label in test_labels])
validation_labels_numerical = np.array([label_mapping[label] for label in validation_labels])
'''

### Arquitecturas sencillas

In [ ]:
#Definimos el tamaño de imagen y el batch size
image_size = (x_size,y_size,3)
batch_size = batch_size

#Definimos el número de clases
num_classes = 3 #Maligno, benigno, normal


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


#Definición de la arquitectura del modelo
#El modelo dependerá del tamaño de la imagen (para la capa de entrada)
# y del número de clases (para la capa de salida)
def get_model(img_size, num_classes):
  #Definimos la entrada
    inputs = keras.Input(shape=img_size)

  #Primera capa convolucional
    x = layers.Rescaling(1./255)(inputs) #reescalar
  #Kernel de tamaño (3, 3)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs) #CHECAR
    #x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    # Segunda capa convolucional
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size= 3, strides=2)(x)

    # Tercera capa convolucional
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    # Cuarta capa convolucional
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    # Aplanar la salida de la última capa de convolución
    x = layers.Flatten()(x)

    # Capa densa completamente conectada
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    # Capa de salida con softmax para clasificación
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Crear el modelo
    model = keras.Model(inputs, outputs)

    return model

# Crear el modelo usando la función
model = get_model(img_size=image_size, num_classes=3)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256, 256, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 63, 63, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 63, 63, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 31, 31, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 31, 31, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 31, 31, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 28800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,686,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,929,155 (14.99 MB)

 Trainable params: 3,928,451 (14.99 MB)

 Non-trainable params: 704 (2.75 KB)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.01), loss="categorical_crossentropy", metrics=["accuracy"])#recall



In [ ]:
history = model.fit(train_dataset,
                    epochs=10,
                    batch_size=batch_size,

                    validation_data=(validation_dataset))


Epoch 1/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.3887 - loss: 23.8406 - val_accuracy: 0.4696 - val_loss: 0.9939
Epoch 2/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.3826 - loss: 1.0986 - val_accuracy: 0.3354 - val_loss: 1.1033
Epoch 3/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.3729 - loss: 1.1184 - val_accuracy: 0.3560 - val_loss: 1.0951
Epoch 4/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.3700 - loss: 1.0979 - val_accuracy: 0.3560 - val_loss: 1.0936
Epoch 5/10
167/460 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.3614 - loss: 1.0970

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "pink", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "pink", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Otra arquitectura


In [ ]:
def get_model2(img_size, num_classes):
  #Definimos la entrada
    inputs = keras.Input(shape=img_size)
    x = layers.Rescaling(1./255)(inputs)

  #Kernel de tamaño (2, 2) para no perder tanta información
  #Ponemos sólo 3 capas convolucionales
  #Agregamos un BatchNormalization

    x = layers.Conv2D(32, 2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    x = layers.Conv2D(64, 2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size= 3, strides=2)(x)

    x = layers.Conv2D(128, 2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)


    x = layers.Flatten()(x)

    #  Dos capas densas y un dropout para mitigar el sobreajuste
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(64, activation='relu')(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)


    model = keras.Model(inputs, outputs)

    return model

# Crear el modelo usando la función
model2 = get_model2(img_size=image_size, num_classes=3)
model2.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 256, 256, 32)        │             416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 256, 256, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 128, 128, 64)        │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 63, 63, 128)         │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 63, 63, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 123008)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │      15,745,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,796,067 (60.26 MB)

 Trainable params: 15,795,619 (60.26 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
#Cambiamos el learning rate
model2.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])#recall

history = model2.fit(train_dataset,
                    epochs=10,
                    batch_size=batch_size,

                    validation_data=(validation_dataset))

Epoch 1/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 23s 34ms/step - accuracy: 0.3923 - loss: 6.6309 - val_accuracy: 0.4450 - val_loss: 1.1683
Epoch 2/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.3695 - loss: 1.1764 - val_accuracy: 0.4346 - val_loss: 1.0954
Epoch 3/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - accuracy: 0.4244 - loss: 1.0721 - val_accuracy: 0.3848 - val_loss: 1.0878
Epoch 4/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - accuracy: 0.3691 - loss: 1.1023 - val_accuracy: 0.4241 - val_loss: 1.0820
Epoch 5/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.3694 - loss: 1.0980 - val_accuracy: 0.3770 - val_loss: 1.0931
Epoch 6/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.3695 - loss: 1.0745 - val_accuracy: 0.3743 - val_loss: 1.1086
Epoch 7/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.3542 - loss: 1.0921 - val_accuracy: 0.4293 - val_loss: 1.2330
Epoch 8/10
460/460 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.3698 - loss: 1.1264 - 

Tercer intento \\
Los valores de perdida tan altos indican que el modelo tiene dificultades para aprender al inicio, esto puede deberse a un mal preprocesamiento de los datos o un problema con la arquitectura del modelo.
Para corregirlo cambiamos Flatten por GlobalAveragePooling2D para tener un tamaño de salida menor.

In [ ]:
def get_model3(img_size, num_classes):
  #Definimos la entrada
    inputs = keras.Input(shape=img_size)
    x = layers.Rescaling(1./255)(inputs)

  #Kernel de tamaño (2, 2) para no perder tanta información
  #Ponemos sólo 3 capas convolucioonales

    x = layers.Conv2D(32, 2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    x = layers.Conv2D(64, 2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size= 3, strides=2)(x)

    x = layers.Conv2D(128, 2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)


    x = layers.GlobalAveragePooling2D()(x)

    #  Dos capas densas y un dropout para mitigar el sobreajuste
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(64, activation='relu')(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)


    model = keras.Model(inputs, outputs)

    return model

# Crear el modelo usando la función
model3 = get_model3(img_size=image_size, num_classes=3)
model3.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 256, 256, 32)        │             416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 256, 256, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 128, 128, 64)        │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 63, 63, 128)         │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 63, 63, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 31, 31, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 67,427 (263.39 KB)

 Trainable params: 66,979 (261.64 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
#Cambiamos el learning rate
model3.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])#recall

history = model3.fit(train_dataset,
                    epochs=20,
                    batch_size=batch_size,

                    validation_data=(validation_dataset))

Epoch 1/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.4464 - loss: 1.0555 - val_accuracy: 0.4785 - val_loss: 0.9845
Epoch 2/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.4934 - loss: 1.0041 - val_accuracy: 0.5215 - val_loss: 0.9490
Epoch 3/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.5005 - loss: 0.9924 - val_accuracy: 0.5116 - val_loss: 0.9586
Epoch 4/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.5057 - loss: 0.9869 - val_accuracy: 0.5403 - val_loss: 0.9354
Epoch 5/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.5204 - loss: 0.9672 - val_accuracy: 0.5313 - val_loss: 0.9184
Epoch 6/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.5293 - loss: 0.9571 - val_accuracy: 0.5492 - val_loss: 0.9099
Epoch 7/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.5200 - loss: 0.9586 - val_accuracy: 0.5394 - val_loss: 0.9107
Epoch 8/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - accuracy: 0.5418 - loss: 0.9385 - 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "pink", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "pink", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Cuarto intento \\
Usamos regularizadores en la arquitectura y callbacks


In [ ]:
from tensorflow.keras import regularizers
def get_model4(img_size, num_classes):
    # Definimos la entrada
    inputs = keras.Input(shape=img_size)
    x = layers.Rescaling(1./255)(inputs)

    # Primera capa convolucional
    x = layers.Conv2D(32, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    # Segunda capa convolucional
    x = layers.Conv2D(64, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    # Tercera capa convolucional
    x = layers.Conv2D(128, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    # Cuarta capa convolucional (opcional)
    x = layers.Conv2D(256, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Capas densas con Dropout y regularización
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Capa de salida
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Crear modelo
    model = keras.Model(inputs, outputs)

    return model

# Crear el modelo
model4 = get_model4(img_size=image_size, num_classes=3)
model4.summary()




In [ ]:
# Compilación del modelo con learning rate más pequeño
from tensorflow.keras.optimizers import Adam

model4.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
# Callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ModelCheckpoint(filepath="best_model4.keras", monitor='val_loss', save_best_only=True)
]

# Entrenamiento
history = model4.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=15,
    batch_size=batchSize,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "pink", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "pink", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

## Aumentado de Datos

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
#Ocupamos el mejor modelo obtenido
def get_model4A(img_size, num_classes):
    # Definimos la entrada
    inputs = keras.Input(shape=img_size)

    x = data_augmentation(inputs)

    x = layers.Rescaling(1./255)(inputs)

    # Primera capa convolucional
    x = layers.Conv2D(32, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    # Segunda capa convolucional
    x = layers.Conv2D(64, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    # Tercera capa convolucional
    x = layers.Conv2D(128, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

    # Cuarta capa convolucional (opcional)
    x = layers.Conv2D(256, 2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Capas densas con Dropout y regularización
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Capa de salida
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Crear modelo
    model = keras.Model(inputs, outputs)

    return model

# Crear el modelo
model4A = get_model4A(img_size=image_size, num_classes=3)
model4A.summary()

In [ ]:
model4A.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])


In [ ]:
# Callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ModelCheckpoint(filepath="best_model4.keras", monitor='val_loss', save_best_only=True)
]

# Entrenamiento
historyA = model4A.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=15,
    batch_size=batchSize,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt
acc = historyA.history["accuracy"]
val_acc = historyA.history["val_accuracy"]
loss = historyA.history["loss"]
val_loss = historyA.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "pink", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "pink", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

## Redes Convolucionales Preentrenadas

### Usaremos Modelo: VGG16

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False, #Quremos llamar la parte de capaz densar? False=no
    input_shape=(x_size, y_size, 3))

In [ ]:
conv_base.summary()

In [ ]:
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)#Preprocesa imagenes para que el modelos las pueda usar !!! Ya no lo programamos nosotros
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
#val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

In [ ]:
train_features.shape

In [ ]:
inputs = keras.Input(train_features.shape[1:])
x = layers.Flatten()(inputs)#Transoformo en vector
x = layers.Dense(512,activation='relu')(x)
x = layers.Dense(256,activation='relu')(x)

x = layers.Dropout(0.8)(x)#Evitar sobreajsute. (elimina neuronas para que no participen) 0.5 = desactiva la mitad de las neuronas
outputs = layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(learning_rate=0.0001),#"rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="breast_tumor_classifier_v1_2024.keras",
      save_best_only=True,
      monitor="val_loss")
]
historyL = model.fit(
    train_features, train_labels,
    epochs=40,
    validation_data=(test_features, test_labels),
    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = historyL.history["accuracy"]
val_acc = historyL.history["val_accuracy"]
loss = historyL.history["loss"]
val_loss = historyL.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "pink", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "pink", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
inputs = keras.Input(train_features.shape[1:])
x = layers.Flatten()(inputs)#Transoformo en vector
x = layers.Dense(512,activation='relu')(x)
x = layers.Dense(256,activation='relu')(x)

x = layers.Dropout(0.8)(x)#Evitar sobreajsute. (elimina neuronas para que no participen) 0.5 = desactiva la mitad de las neuronas
outputs = layers.Dense(3, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.0001),
    metrics=["accuracy"]
)

# Callbacks
callbacks = [
    ModelCheckpoint(filepath="best_model_v2.keras", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

# Entrenamiento
history = model.fit(
    train_features, train_labels,
    epochs=40,
    validation_data=(test_features, test_labels),
    callbacks=callbacks,
    batch_size=32
)

In [ ]:
import matplotlib.pyplot as plt
acc = historyL.history["accuracy"]
val_acc = historyL.history["val_accuracy"]
loss = historyL.history["loss"]
val_loss = historyL.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "pink", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "pink", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Usando Aumentado de Datos

In [ ]:
#FORMA MAS EFICIENTE DE HACERLO
from tensorflow.keras.applications import ResNet50
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
inputs = keras.Input(shape=(x_size,y_size,3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x) #este aumento de datos los paso por el proceso de preprocesamiento vgg16
x = conv_base(x)
x=layers.Flatten()(x)
#x = GlobalAveragePooling2D()(x)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(0.5)(x)#Evitar sobreajsute. (elimina neuronas para que no participen) 0.5 = desactiva la mitad de las neuronas
outputs = layers.Dense(3, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.0001),#"rmsprop",
    metrics=["accuracy"]
)
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="breast_tumor_classifier_v1_2024.keras",
        save_best_only=True,
        monitor="val_loss")
]

# Entrenamiento
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    batch_size=4,
    callbacks=callbacks)

Usando ResNet50

In [ ]:
'''
conv_base2 = keras.applications.ResNet50(
    weights="imagenet",
    include_top=False, #Quremos llamar la parte de capaz densar? False=no
    classifier_activation = "sigmoid",
    input_shape=(x_size, y_size, 3))
 '''

In [ ]:
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.resnet.preprocess_input(images)#Preprocesa imagenes para que el modelos las pueda usar !!! Ya no lo programamos nosotros
        features = conv_base2.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
#val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

In [ ]:
inputs = keras.Input(train_features.shape[1:])
x = layers.Flatten()(inputs)#Transoformo en vector
x = layers.Dense(512,activation='relu')(x)
x = layers.Dense(256,activation='relu')(x)


x = layers.Dropout(0.8)(x)#Evitar sobreajsute. (elimina neuronas para que no participen) 0.5 = desactiva la mitad de las neuronas
outputs = layers.Dense(3, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

# Callbacks
callbacks = [
    ModelCheckpoint(filepath="breast_tumor_classifier_v1_2024.keras", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

# Entrenamiento
history = model.fit(
    train_features, train_labels,
    epochs=40,
    validation_data=(test_features, test_labels),
    callbacks=callbacks,
    batch_size=32
)